# Decision Tree

In this notebook, we will introduce one of the most popular supervised machine learning algorithms called decision tree. It can be used for both regression and classification algorithm. Here we mainly discuss decision tree from classification perspective.

In [1]:
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

from abc import ABCMeta, abstractclassmethod

%matplotlib inline

## Introduction 

<p><center>![Basic Tree Terminology](../../images/inkscape/basic_tree.svg)</center></p>
<p><center><b>Figure 1</b>: Basic Tree Terminology</center></p>


## Building Tree Model

<p><center>![Basic Tree Terminology](../../images/inkscape/tree_building.svg)</center></p>
<p><center><b>Figure 2</b>: Basic Tree Terminology</center></p>


## Splitting Data into Different Regions

<p><center>![Basic Tree Terminology](../../images/inkscape/tree_partition.svg)</center></p>
<p><center><b>Figure 2</b>: Basic Tree Terminology</center></p>

## How to Choose Best Split?

## Decision Tree Algorithm

$$H(S)=-\sum_{c \in C}p(c)log_2\big(p(c)\big)$$

In [2]:
def entropy(x):
    # remove all zero and negative values
    x = x[x > 0]
    return -np.sum(x * np.log2(x))

$$I = H(S) - \sum_{i \in (1, 2)}\frac{S^{i}}{S}H(S^{i})$$

In [3]:
def expected_entropy(s_i, s_size):
    s_i_size = s_i.shape[0]
    if s_i_size == 0:
        return 0.0
    return ((s_i_size) / s_size) * entropy(s_i)
    
def information_gain(x, cutoff_index):
    x_size = x.shape[0]
    left = expected_entropy(x[:cutoff_index], x_size)
    right = expected_entropy(x[cutoff_index:], x_size)
    return entropy(x) - (left + right)

In [4]:
class Node(object):
    def __init__(self, id=None, description=None):
        self._id = id
        self._description = description
        
    @property
    def id(self):
        return self._id
    
    @id.setter
    def id(self, value):
        self._id = value
        
    @property
    def description(self):
        return self._description
    
    @description.setter
    def description(self, value):
        self._description = value  
        
class Leaf(Node):
    def __init__(self, values, n_classes, id=None, description=None):
        Node.__init__(id, description)
        self._values = values
        
    @property
    def values(self):
        return self._values
    
    @values.setter
    def values(self, value):
        self._values = value
        
class Internal(Node):
    def __init__(self, dim, threshold, left_child, right_child, node_id=None, description=None):
        Node.__init__(id, description)
        self._dim = dim
        self._threshold = threshold
        self._left_child = left_child
        self._right_child = right_child
        
    @property
    def dim(self):
        return self._dim
        
    @dim.setter
    def dim(self, value):
        self._dim = value
        
    @property
    def threshold(self):
        return self._threshold
    
    @threshold.setter
    def threshold(self, value):
        self._threshold = threshold
        
    @property
    def left_child(self):
        return self._left_child
    
    @left_child.setter
    def left_child(self, value):
        self._left_child = value
        
    @property
    def right_child(self):
        return self._right_child
    
    @right_child.setter
    def right_child(self, value):
        self._right_child = value

In [5]:
class BaseTree(metaclass=ABCMeta):
    def __init__(self, max_depth=None, n_min_leaf=2, n_trials=None):
        self._max_depth = max_depth
        self._n_min_leaf = n_min_leaf
        self._n_trials = n_trials
        
        self._root_node = None 
        self._num_classes = None # delete ?
        
    @property
    def max_depth(self):
        return self._max_depth
    
    @max_depth.setter
    def max_depth(self, value):
        self._max_depth = value
        
    @property
    def n_min_leaf(self):
        self._n_min_leaf
        
    @n_min_leaf.setter
    def n_min_leaf(self, value):
        self._n_min_leaf = value
        
    @property
    def n_trials(self):
        return self._n_trials
    
    @n_trials.setter
    def n_trials(self, value):
        self._n_trials = value   
    
    def fit(self, X_train, y_train):
        pass
    
    def prefict(self, X_test):
        pass    
    
    def _fit(self, X, y, max_depth=None, n_min_leaf=None, n_trails=None):
        n_classes = np.unique(y).shape[0
                                      ]
        # if all data points are in the same class, creae a leaf
        if np.all(y == y[0]):
            return Leaf(y, n_classes)
        
        # if we have reached the max_depth, create a leaf
        if max_depth is not None and max_depth <= 0:
            return Leaf(y, n_classes)
        
        # Now we have to split the current node
        # First calculate split parameters
        
    
    def _split_parameters(self, X, Y, n_min_leaf=None, n_trials=None):
        pass
    
    @abstractmethod
    def _find_best_split_threshold(self, feature_vector, response):
        pass

    @abstractmethod
    def _predict_single_data_point(self, X, node, emit_probability=False):
        pass

NameError: name 'abstractmethod' is not defined

## References and Further Reading

1. https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/decisionForests_MSR_TR_2011_114.pdf [Decision Forests for Classification, Regression, Density Estimation, Manifold Learning and Semi-Supervised Learning]

2. 